In [1]:
import os
import json
import copy
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from tools.maneger import DataManager
from tools.preproc import Preprocessor
# from utterance.feature import Feature
from feature import Feature

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
from pyknp import Juman
from sentence_transformers import SentenceTransformer
import scipy.spatial
model_path = "/home/yamada/Downloads/training_bert_japanese"
Nmodel = SentenceTransformer(model_path, show_progress_bar=False)
emb_dim = Nmodel.encode(["お辞儀をしている男性会社員"])[0].shape[0]

In [4]:
pre = Preprocessor()

300


In [5]:
F_path = "../X_y_data/response/"
F_name = "typeClassify_F.dill"
featureM = DataManager(F_path, format_="dill")

Fe = featureM.load_data(F_name)
Fe.set_preprocessor(pre)

success load : ../X_y_data/response/typeClassify_F.dill


In [6]:
model_path = "../models/response/"
model_name = "typeClassify_M.pickle"
modelM = DataManager(model_path)
LR = modelM.load_data(model_name)

success load : ../models/response/typeClassify_M.pickle


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import loss
import torch.optim as optim

torch.cuda.is_available()

True

In [8]:
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, tagset_size, batch_size):
        # 親クラスのコンストラクタ。決まり文句
        super(LSTMClassifier, self).__init__()
        # 隠れ層の次元数。これは好きな値に設定しても行列計算の過程で出力には出てこないので。    
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        # LSTMの隠れ層。これ１つでOK。超便利。
        self.lstm = nn.LSTM(embedding_dim, hidden_dim//2, batch_first=True, bidirectional=True )
        # LSTMの出力を受け取って全結合してsoftmaxに食わせるための１層のネットワーク
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        # softmaxのLog版。dim=0で列、dim=1で行方向を確率変換。
        # self.softmax = 
    
    def forward(self, x):
        #embeds.size() = (batch_size × len(sentence) × embedding_dim)
        batch_size, seq_len = x.shape[0], x.shape[1]
        _, hidden_layer = self.lstm(x)
        # print(hidden_layer)
        bilstm_out = torch.cat([hidden_layer[0][0], hidden_layer[0][1]], dim=1)
        # y = self.hidden2tag(hidden_layer[0].view(batch_size, -1))

        y = self.hidden2tag(bilstm_out)
        y = F.log_softmax(y, dim=1)
        return y

In [9]:
Model_path = "../models/response/"
Model_name = "question1_M.pickle"
ModelM = DataManager(model_path)
print(Model_name)
Model = ModelM.load_data(Model_name)

question1_M.pickle
success load : ../models/response/question1_M.pickle


In [10]:
path = '../error_category_classification/dbdc5_ja_dev_labeled/'
datalist = ['DCM', 'DIT', 'IRS']
# datalist = ['DCM']
    # List of error types
error_types = ['Unclear intention', 'Wrong information',
    'Ignore question', 'Topic transition error', 
    'Lack of information', 'Repetition', 
    'Contradiction', 'Self-contradiction',
    'Lack of common sense', 'Semantic error',
    'Grammatical error', 'Ignore proposal', 
    'Ignore offer', 'Lack of sociality', 
    'Uninterpretable', 'Ignore greeting', 
    'No-Err']
df = pre.read_json_with_NoErr(path, datalist)

In [11]:
def make_X_str_y(df, errors):
    X_str = []
    y = []
    y = np.zeros(len(df))
    for i, (u, s, e) in enumerate(zip(df.usr, df.sys, df.ec)):
        # sentence_vectors = Nmodel.encode([u, s])
        X_str.append([u, s])
        for err in errors:
            if err in e:
                y[i] = 1

    return X_str, y

In [19]:
errors = ["Ignore question"]

In [18]:
Model.to("cpu")

LSTMClassifier(
  (lstm): LSTM(768, 1536, batch_first=True, bidirectional=True)
  (hidden2tag): Linear(in_features=3072, out_features=2, bias=True)
)

In [13]:
X_str, y = make_X_str_y(df, errors)

In [14]:
def make_X(X_str, Fe: Feature, lr, Model, lr_mode="proba", mode="dif"):
    X = []
    doc_vectors = []
    # doc_vectors = []
    for conv in X_str:
        sentence_vectors = Nmodel.encode(conv)
        doc_vectors.append(sentence_vectors)
        diff = sentence_vectors[0] - sentence_vectors[1]
        if np.linalg.norm(diff) == 0:
            diff_norm = diff
        else:
            diff_norm = diff/np.linalg.norm(diff)
        # ユーザ発話の特徴
        f = Fe.featurization(conv[0])
        if lr_mode == "proba":
            p = lr.predict_proba(f.reshape(1,-1))
            is_ques = p[0]
        elif lr_mode == "pred":
            p = lr.predict(f.reshape(1,-1))
            if p == 0 or p == 1 :
                is_ques = np.array([1])
            else:
                is_ques = np.array([0])
        
        X.append( np.concatenate([diff_norm, is_ques]) )
    
    X = np.array(X)
    # 後ろ向き発話具合
    with torch.no_grad():
        # for i  in range
        # back_x = torch.tensor( doc_vectors , device='cuda:0').float()
        back_x = torch.tensor( doc_vectors).float()
        back_y = np.array(Model(back_x)).argmax(axis=1)
    
    X = np.block([X, back_y.reshape(-1, 1)])
    del doc_vectors
    return np.asarray(np.nan_to_num(X), dtype=np.float32)


In [20]:

X = make_X(X_str, Fe, LR, Model, lr_mode="pred")

In [21]:
data_path = "../X_y_data/response/"
data_name = "question_D1.pickle"
dataM = DataManager(data_path)
print(data_name)

question_D1.pickle


In [ ]:
# if dataM.is_exist(data_name):
#     DATA_Xy = dataM.load_data(data_name)
#     X = DATA_Xy[0]
#     y = DATA_Xy[1]

In [22]:
dataM.save_data(data_name, [X, y])

success save : ../X_y_data/response/question_D1.pickle


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=5)

In [ ]:
leng = len(y_train)
print(leng)
for i, v in enumerate(y_train):
    if leng %(i+1) == 0:
        print(i+1, end=", ")

In [24]:
lr = LogisticRegression(solver='sag', max_iter=1000)
lr.fit(X_train, y_train)

LogisticRegression(max_iter=1000, solver='sag')

In [25]:
y_pred = lr.predict(X_test)

In [26]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))

confusion matrix = 
 [[328  18]
 [ 22  32]]
accuracy =  0.9


confusion matrix = 
 [[332  14]
 [ 33  21]]
accuracy =  0.8825

In [33]:
correct_n = 0
all_tp_one = 0
for t, n in zip(y_test, y_pred):
    if t == 1:
        all_tp_one += 1
        if n == 1:
            correct_n += 1
print("corrent_n:", correct_n)
print("all_tp_one:", all_tp_one)
print("rate c:", correct_n/all_tp_one)
print("rate bad:",  (len(y_pred)-all_tp_one)/len(y_pred))

corrent_n: 32
all_tp_one: 54
rate c: 0.5925925925925926
rate bad: 0.865


In [ ]:
# def make_X2(X_str, Fe: Feature, lr , lr_mode="proba"):
#     X = []
#     # doc_vectors = []
#     for conv in X_str:
#         sentence_vectors = Nmodel.encode(conv)
#         doc_vectors.append(sentence_vectors)
#         diff = sentence_vectors[0] - sentence_vectors[1]
#         if np.linalg.norm(diff) == 0:
#             diff_norm = diff
#         else:
#             diff_norm = diff/np.linalg.norm(diff)
#         # ユーザ発話の特徴
#         f = Fe.featurization(conv[0])
#         if lr_mode == "proba":
#             is_ques = lr.predict_proba(f.reshape(1,-1))
#         elif lr_mode == "pred":
#             is_ques = lr.predict(f.reshape(1,-1))
        
#         X.append( np.concatenate([diff_norm, is_ques]) )

In [28]:
leng = len(df)
y_rule = np.zeros(leng)
users_f =[ Fe.featurization(t) for t in X_str[:leng]]
is_ques = LR.predict(users_f)
vec_test = [ Nmodel.encode(t) for t in X_str[:leng]]
with torch.no_grad():
    back_x = torch.tensor( vec_test).float()
    back_y = np.array(Model(back_x)).argmax(axis=1)

In [29]:
for i, (q, b) in enumerate(zip(is_ques, back_y)):
    if (q==0 or q == 1) and b == 1:
        y_rule[i] = 1

In [30]:
print('confusion matrix = \n', confusion_matrix(y_true=y[:leng], y_pred=y_rule))
print('accuracy = ', accuracy_score(y_true=y[:leng], y_pred=y_rule))

confusion matrix = 
 [[1544  150]
 [ 124  182]]
accuracy =  0.863


confusion matrix = 
 [[1510  184]
 [ 149  157]]
accuracy =  0.8335

In [32]:
correct_n = 0
all_tp_one = 0
for t, n in zip(y[:leng], y_rule):
    if t == 1:
        all_tp_one += 1
        if n == 1:
            correct_n += 1
print("corrent_n:", correct_n)
print("all_tp_one:", all_tp_one)
print("rate c:", correct_n/all_tp_one)
print("rate bad:",  (len(y_rule)-all_tp_one)/len(y_rule))

corrent_n: 182
all_tp_one: 306
rate c: 0.5947712418300654
rate bad: 0.847


corrent_n: 157
all_tp_one: 306
rate c: 0.5130718954248366
rate bad: 0.235